# Single Node Single GPU Training

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import dask

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [3]:
model_params = {'n_epochs': 50, 
                'base_lr': .02,
               'batchsize': 64}

wbargs = {**model_params,
    'classes':10,
    'Notes':"tf_v100_8x",
    'Tags': ['single', 'gpu', 'tensorflow'],
    'dataset':"Clothing",
    'architecture':"ResNet50"}

In [4]:

# USE categorical_crossentropy
# tester_flow = image.ImageDataGenerator()

#     tester_flow = image.ImageDataGenerator()
#     train_ds = tester_flow.flow_from_directory(
#         directory = '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
#         target_size=(224,224),
#         batch_size=batchsize
#     )

#     valid_ds = tester_flow.flow_from_directory(
#         directory = '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/validation',
#         target_size=(224,224),
#         batch_size=batchsize
#     )

################

# USE sparse_categorical_crossentropy
# # Data
# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     '/home/jovyan/git-repos/snowflake-ml-guide/data/clothing-dataset-small/train',
#     seed=123,
#     image_size=(224,224),
#     batch_size=32).prefetch(2)

In [5]:
def train_model_fit(n_epochs, base_lr, batchsize):
    
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        classes=10)
        
    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='tensorflow_blog')
    config=wbargs
    
    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/tensorflow_ds/datasets/birds/285 birds/train',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2).cache().shuffle(1000)
        
    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/tensorflow_ds/datasets/birds/285 birds/valid',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2)
   
    optimizer = keras.optimizers.Adam(lr=base_lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #integer labels

    outputs = model.fit(
        train_ds, 
        verbose = 1, 
        epochs=n_epochs, 
        validation_data=valid_ds,
        callbacks=[WandbCallback()]
    )
    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras_single/')

In [6]:
tester = train_model_fit(**model_params)

Found 3068 files belonging to 10 classes.
Found 341 files belonging to 10 classes.
Epoch 1/50
48/48 [==============================] - 20s 250ms/step - loss: 9.5674 - accuracy: 0.1635 - val_loss: 111369150464.0000 - val_accuracy: 0.2375
Epoch 2/50
48/48 [==============================] - 11s 209ms/step - loss: 2.1138 - accuracy: 0.2629 - val_loss: 190574.4219 - val_accuracy: 0.0528
Epoch 3/50
48/48 [==============================] - 11s 209ms/step - loss: 2.0449 - accuracy: 0.2937 - val_loss: 746.4316 - val_accuracy: 0.2170
Epoch 4/50
48/48 [==============================] - 11s 208ms/step - loss: 1.9332 - accuracy: 0.3433 - val_loss: 39.7087 - val_accuracy: 0.2493
Epoch 5/50
48/48 [==============================] - 11s 209ms/step - loss: 1.8895 - accuracy: 0.3378 - val_loss: 3.9158 - val_accuracy: 0.2933
Epoch 6/50
48/48 [==============================] - 11s 209ms/step - loss: 1.8476 - accuracy: 0.3578 - val_loss: 2.8575 - val_accuracy: 0.3021
Epoch 7/50
48/48 [======================

epoch,49
loss,0.07518
accuracy,0.97718
val_loss,2.86153
val_accuracy,0.54839
_runtime,591
_timestamp,1629381074
_step,49
best_val_loss,1.36058
best_epoch,17


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████████████
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▃▁▃▃▄▃▄▄▄▄▅▅▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇█▇▇▇▇█▇█▇█▇▆
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


INFO:tensorflow:Assets written to: model/keras_single/assets
